In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from feature_engine.imputation import MeanMedianImputer

from catboost import CatBoostRegressor, Pool, CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import r2_score, f1_score

from pycaret.regression import *
from IPython import display
display.set_matplotlib_formats('svg')

In [2]:
df = pd.read_csv('HDD_wasabiV7_G7.csv')
df.shape

(2007, 678)

In [3]:
df.head()

,Freq,pACC_mean,pACC_median,delta_acc_mean,delta_acc_median,mr_res_hsa_P5,mr_res_hsa_P95,mr_res_hsa_median,mr_res_hsa_std,tfc_res_hsa_P5,...,AD_Y_Ovly_8410,AD_Y_Ovly_8410_Sigma,FLMILL_THK_SCD_4037,FLMILL_THK_SCD_4037_Sigma,K3R3_CALC_2288,K3R3_CALC_sigma_2288,P3TABS_PRED_7162,AL2O3TADJ_2722,Delta_H90_H10_L,Delta_H90_H10_U
0,1365,102.551041,102.512528,0.177232,0.156243,350.900,555.7100,445.55,64.300704,62.59,...,NaN,NaN,337.823486,2.973033,NaN,NaN,55.619469,311.843994,-927.326740,-79.505876
1,1802,101.998499,102.005471,-0.328914,-0.305854,320.760,487.4375,399.50,51.707776,62.59,...,-112.5437,3.325268,NaN,NaN,2.228111,2.538262,51.781448,306.458313,-1391.511243,-695.033075
2,1236,101.362012,101.485516,-1.047037,-0.902902,312.300,480.3125,398.00,51.515879,62.65,...,-115.6329,3.077855,NaN,NaN,1.464920,1.828405,51.541012,302.358612,-1292.631784,-699.773161
3,947,101.903842,101.919784,-0.602042,-0.622663,325.114,492.5000,404.00,52.443014,62.06,...,-115.2511,3.124967,NaN,NaN,-2.747694,1.869439,51.828239,301.805695,-1400.488076,-550.858628
4,1682,101.605268,101.640547,-0.722127,-0.660263,332.500,508.6875,415.00,54.150842,62.13,...,-112.2895,3.716977,NaN,NaN,0.353722,2.100429,54.262600,306.233887,-1412.538728,-555.741211


In [4]:
X_drop = ['WAFERNUM', 'Freq', 'pACC_median', 'pACC_mean', 'delta_acc_mean','delta_acc_median',
         'u100_ser_62_P5','u30n_ser_00_P95','u30n_ser_31_P95','u101_ser_62_median','u30n_ser_31_P5','u101_ser_00_P95',
          'u30n_ser_31_median','u101_ser_00_P5','u100_ser_31_std','u30n_ser_62_P95','u100_ser_31_median','u101_ser_00_std',
          'u100_ser_00_median','u100_ser_31_P95','u101_ser_31_P95','u101_ser_31_median','u101_ser_31_std',
          'u30n_ser_00_median','u101_ser_62_P5','u30n_ser_62_P5','u100_ser_62_median','u101_ser_62_P95',
          'u101_ser_00_median','u100_ser_31_P5','u101_ser_31_P5','u30n_ser_00_std','u30n_ser_62_std','u100_ser_00_P95',
          'u30n_ser_31_std','u100_ser_62_std','u100_ser_00_P5','u101_ser_62_std','u100_ser_62_P95','u100_ser_00_std',
          'u100_tpi_numcusttrks_std', 'u100_tpi_numcusttrks_median','u100_tpi_numcusttrks_P5','u100_tpi_numcusttrks_P95',
          'u30n_ser_62_median']

X_train, X_test, y_train, y_test = train_test_split(df.drop(X_drop, axis=1),
                                              df['delta_acc_mean'],
                                              test_size = 0.3,
                                              random_state = 1)

X_train.shape, X_test.shape

((1404, 633), (603, 633))

### data cleaning and imputation

In [5]:
missing_var = [var for var in X_train.columns.tolist() if X_train[var].isnull().mean() > 0.5]
len(missing_var)

0

In [6]:
constant_var = [var for var in X_train.columns if X_train[var].std()==0]
len(constant_var)

0

In [7]:
# find feature names for missing data
all_cols = [c for c in X_train.columns.tolist()]
missing_f_train = []
missing_f_test = []
missing_f_val = []

for col in all_cols:
    if X_train[col].isnull().sum() > 0:
        missing_f_train.append(col)
        print('column : {} | Nulls : {}'.format(col, X_train[col].isnull().sum()))
        
print('='*50)
        
for col in all_cols:
    if X_test[col].isnull().sum() > 0:
        missing_f_test.append(col)
        print('column : {} | Nulls : {}'.format(col, X_test[col].isnull().sum()))

column : DVH11_H1_RMin_32639 | Nulls : 533
column : DVH11_H1_RMin_32639_Sigma | Nulls : 533
column : ETWL_33557 | Nulls : 665
column : ETWL_33557_Sigma | Nulls : 665
column : ETWU_33557 | Nulls : 665
column : ETWU_33557_Sigma | Nulls : 665
column : TMR4UR_33557 | Nulls : 665
column : TMR4UR_33557_Sigma | Nulls : 665
column : TMR4LR_33557 | Nulls : 665
column : TMR4LR_33557_Sigma | Nulls : 665
column : KTCRAL_33557 | Nulls : 665
column : KTCRAU_33557 | Nulls : 665
column : A1LWINDAGE_33557 | Nulls : 665
column : A1UWINDAGE_33557 | Nulls : 665
column : RA4LR_33557 | Nulls : 665
column : RA4LR_33557_Sigma | Nulls : 665
column : RA4UR_33557 | Nulls : 665
column : RA4UR_33557_Sigma | Nulls : 665
column : TMRX1L_33557 | Nulls : 665
column : TMRX1L_33557_Sigma | Nulls : 665
column : TMRX1U_33557 | Nulls : 665
column : TMRX1U_33557_Sigma | Nulls : 665
column : A1L_HEFFECTIVE_33564 | Nulls : 659
column : A1L_HEFFECTIVE_33564_Sigma | Nulls : 659
column : A1U_HEFFECTIVE_33564 | Nulls : 659
column

In [8]:
# get all missing feature names
missing_f = missing_f_train + missing_f_test
missing_f = list(set(missing_f))

len(missing_f)

97

In [9]:
median_imputer = MeanMedianImputer(imputation_method='median', variables=missing_f)
median_imputer.fit(X_train)

MeanMedianImputer(variables=['SS2P3A53_7187', 'A1U_A1ASYMMETRY_33564',
                             'A6MILLNANO_THK_5447_Sigma', 'TWPP3A_7187',
                             'TT_Y_ovly_7183_Sigma', 'H90_ELEMU_33558',
                             'H50_ELEML_33558_Sigma', 'ETWU_33557',
                             'DVH12_ElemU_HMaxAP1_33558',
                             'A6CMPFIB_A6THK_8973_Sigma', 'TMRX1U_33557',
                             'A1L_HEFFECTIVE_33564', 'A6MILLNANO_THK_5447',
                             'FD_Atlas_MCD_6667_Sigma', 'TMR4LR_33557_Sigma',
                             'A1U_AMPLOWFIELD_33564_Sigma', 'AL2O3T_ADJ_2819',
                             'H10_ELEMU_33558_Sigma', 'A1U_HEFFECTIVE_33564',
                             'H50_ELEMU_33558', 'A1UWINDAGE_33557',
                             'Final_P3A_7187_Sigma', 'HMAXAP1_ELEML_33558',
                             'A6_OVERLAY_7525_Sigma', 'TMRX1L_33557',
                             'TMR4UR_33557', 'AL2O3TADJ_272

In [10]:
X_train = median_imputer.transform(X_train)
X_test = median_imputer.transform(X_test)

X_train.isnull().sum().sum(), X_test.isnull().sum().sum()

(0, 0)

### Feature Selection

**1. SelectFromModel**

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel

sel_ = SelectFromModel(GradientBoostingRegressor())
sel_.fit(X_train, y_train)

selected_feat = X_train.columns[sel_.get_support()]
len(selected_feat), selected_feat

(67, Index(['u101_mrwid_P5', 'u101_mrwid_median', 'u101_mrwod_P95',
        'u31n_owpav1md_P95', 'u105_cts_md_median', 'u100_snr_id_P5',
        'u300_snr_md_P5', 'u300_snr_md_median', 'u300_snr_id_P5',
        'u300_snr_id_P95', 'u300_snr_id_median', 'u300_snr_id_std',
        'u301_snr_od_P95', 'u301_snr_md_P5', 'u301_snr_md_P95',
        'u301_snr_md_median', 'u301_snr_md_std', 'u301_snr_id_P5',
        'u301_snr_id_median', 'u301_snr_id_std', 'u100_res1od_P5',
        'u100_res1od_median', 'u100_res1md_P5', 'u100_res1md_median',
        'u100_res1id_P5', 'u100_res1id_median', 'u101_res1md_P95',
        'u101_res1id_median', 'u10n_ivc_kc_08_std', 'u10n_ivc_oiv_00_std',
        'u10n_ivc_oiv_01_P5', 'u10n_ivc_oiv_01_P95', 'u10n_ivc_oiv_01_median',
        'u10n_ivc_oiv_02_median', 'u10n_ivc_oiv_02_std', 'u10n_ivc_oiv_03_P95',
        'u10n_ivc_oiv_03_median', 'u10n_ivc_oiv_04_P95',
        'u10n_ivc_oiv_04_median', 'u10n_ivc_oiv_05_P5', 'u10n_ivc_oiv_07_P95',
        'u10n_ivc_oiv_07

**2. Boruta**

In [14]:
from sklearn.ensemble import GradientBoostingRegressor
from boruta import BorutaPy

boruta = BorutaPy(estimator = GradientBoostingRegressor(),
                 n_estimators = 'auto',
                 max_iter = 10)
boruta.fit(X_train.values, y_train.values.ravel())

boruta_support = X_train.columns[boruta.support_].to_list()
boruta_support_weak = X_train.columns[boruta.support_weak_].to_list()

In [17]:
len(boruta_support), len(boruta_support_weak)

(42, 19)

In [24]:
selected_feat = boruta_support

### Modelling

In [12]:
import warnings 
warnings.filterwarnings("ignore")

# 一个训练优化模型的函数
def train_optimal_classifier(clf,param):
    grid_search = GridSearchCV(
        clf,
        param_grid=param,
        cv=5,
        verbose=0,
        scoring='r2',
        error_score=0,
        refit=True
        )
    grid_search.fit(X_train[selected_feat], y_train)
    print("Best parameters")
    print(grid_search.best_params_)
    print('best R2: ')
    print(grid_search.best_score_)
    return (grid_search.best_estimator_)

In [13]:
cat_cv = CatBoostRegressor()
param_dic = {'depth': [3, 5, 7],
             'learning_rate': [0.1, 0.5],
             'iterations': [50, 100, 500],
             'l2_leaf_reg': [1, 2, 3]
             }
train_optimal_classifier(cat_cv,param_dic)

0:	learn: 0.4461674	total: 179ms	remaining: 8.79s
1:	learn: 0.4226389	total: 187ms	remaining: 4.48s
2:	learn: 0.4021984	total: 191ms	remaining: 3s
3:	learn: 0.3842192	total: 195ms	remaining: 2.25s
4:	learn: 0.3699980	total: 201ms	remaining: 1.81s
5:	learn: 0.3567257	total: 243ms	remaining: 1.78s
6:	learn: 0.3457358	total: 246ms	remaining: 1.51s
7:	learn: 0.3357631	total: 250ms	remaining: 1.31s
8:	learn: 0.3266526	total: 254ms	remaining: 1.16s
9:	learn: 0.3187246	total: 258ms	remaining: 1.03s
10:	learn: 0.3122786	total: 262ms	remaining: 930ms
11:	learn: 0.3058543	total: 266ms	remaining: 843ms
12:	learn: 0.2995642	total: 270ms	remaining: 767ms
13:	learn: 0.2950045	total: 294ms	remaining: 756ms
14:	learn: 0.2899041	total: 298ms	remaining: 695ms
15:	learn: 0.2845713	total: 301ms	remaining: 640ms
16:	learn: 0.2801009	total: 304ms	remaining: 590ms
17:	learn: 0.2755591	total: 308ms	remaining: 548ms
18:	learn: 0.2715978	total: 350ms	remaining: 571ms
19:	learn: 0.2675547	total: 359ms	remaining:

18:	learn: 0.2669940	total: 254ms	remaining: 414ms
19:	learn: 0.2639105	total: 257ms	remaining: 385ms
20:	learn: 0.2606431	total: 260ms	remaining: 359ms
21:	learn: 0.2578043	total: 296ms	remaining: 376ms
22:	learn: 0.2543529	total: 300ms	remaining: 353ms
23:	learn: 0.2510479	total: 304ms	remaining: 329ms
24:	learn: 0.2479377	total: 307ms	remaining: 307ms
25:	learn: 0.2451075	total: 311ms	remaining: 287ms
26:	learn: 0.2428338	total: 314ms	remaining: 268ms
27:	learn: 0.2407284	total: 318ms	remaining: 249ms
28:	learn: 0.2388432	total: 321ms	remaining: 232ms
29:	learn: 0.2364033	total: 324ms	remaining: 216ms
30:	learn: 0.2342266	total: 342ms	remaining: 209ms
31:	learn: 0.2321867	total: 345ms	remaining: 194ms
32:	learn: 0.2311416	total: 348ms	remaining: 179ms
33:	learn: 0.2296226	total: 402ms	remaining: 189ms
34:	learn: 0.2278583	total: 406ms	remaining: 174ms
35:	learn: 0.2265784	total: 428ms	remaining: 167ms
36:	learn: 0.2249415	total: 453ms	remaining: 159ms
37:	learn: 0.2231975	total: 456

0:	learn: 0.3609416	total: 4.21ms	remaining: 206ms
1:	learn: 0.3143591	total: 7.47ms	remaining: 179ms
2:	learn: 0.2911401	total: 10.6ms	remaining: 166ms
3:	learn: 0.2705957	total: 13.7ms	remaining: 158ms
4:	learn: 0.2575012	total: 16.9ms	remaining: 152ms
5:	learn: 0.2483194	total: 20.1ms	remaining: 147ms
6:	learn: 0.2415729	total: 23.2ms	remaining: 142ms
7:	learn: 0.2326532	total: 46ms	remaining: 242ms
8:	learn: 0.2272956	total: 49.5ms	remaining: 225ms
9:	learn: 0.2234863	total: 52.9ms	remaining: 212ms
10:	learn: 0.2201691	total: 99.9ms	remaining: 354ms
11:	learn: 0.2159918	total: 103ms	remaining: 327ms
12:	learn: 0.2120225	total: 107ms	remaining: 304ms
13:	learn: 0.2083707	total: 112ms	remaining: 287ms
14:	learn: 0.2046668	total: 115ms	remaining: 269ms
15:	learn: 0.2007934	total: 119ms	remaining: 252ms
16:	learn: 0.1964675	total: 122ms	remaining: 237ms
17:	learn: 0.1939771	total: 126ms	remaining: 224ms
18:	learn: 0.1909015	total: 161ms	remaining: 263ms
19:	learn: 0.1871340	total: 165m

23:	learn: 0.2562443	total: 192ms	remaining: 208ms
24:	learn: 0.2531845	total: 197ms	remaining: 197ms
25:	learn: 0.2509613	total: 201ms	remaining: 185ms
26:	learn: 0.2485919	total: 205ms	remaining: 175ms
27:	learn: 0.2468661	total: 209ms	remaining: 164ms
28:	learn: 0.2439484	total: 247ms	remaining: 179ms
29:	learn: 0.2413050	total: 250ms	remaining: 167ms
30:	learn: 0.2395074	total: 253ms	remaining: 155ms
31:	learn: 0.2376574	total: 256ms	remaining: 144ms
32:	learn: 0.2362495	total: 260ms	remaining: 134ms
33:	learn: 0.2344569	total: 263ms	remaining: 124ms
34:	learn: 0.2332405	total: 266ms	remaining: 114ms
35:	learn: 0.2317844	total: 270ms	remaining: 105ms
36:	learn: 0.2296220	total: 273ms	remaining: 95.9ms
37:	learn: 0.2281534	total: 300ms	remaining: 94.8ms
38:	learn: 0.2265825	total: 304ms	remaining: 85.7ms
39:	learn: 0.2252935	total: 308ms	remaining: 77ms
40:	learn: 0.2240200	total: 312ms	remaining: 68.5ms
41:	learn: 0.2226826	total: 316ms	remaining: 60.2ms
42:	learn: 0.2211431	total:

0:	learn: 0.4425793	total: 7.02ms	remaining: 344ms
1:	learn: 0.4198824	total: 13.2ms	remaining: 316ms
2:	learn: 0.4000034	total: 16.5ms	remaining: 258ms
3:	learn: 0.3832956	total: 19.8ms	remaining: 228ms
4:	learn: 0.3695114	total: 23ms	remaining: 207ms
5:	learn: 0.3569862	total: 54.7ms	remaining: 401ms
6:	learn: 0.3452628	total: 58.2ms	remaining: 358ms
7:	learn: 0.3352846	total: 61.8ms	remaining: 325ms
8:	learn: 0.3262013	total: 65.2ms	remaining: 297ms
9:	learn: 0.3183287	total: 68.7ms	remaining: 275ms
10:	learn: 0.3113570	total: 72.3ms	remaining: 256ms
11:	learn: 0.3056665	total: 75.6ms	remaining: 239ms
12:	learn: 0.2990234	total: 78.8ms	remaining: 224ms
13:	learn: 0.2936458	total: 110ms	remaining: 283ms
14:	learn: 0.2890839	total: 114ms	remaining: 265ms
15:	learn: 0.2835579	total: 117ms	remaining: 249ms
16:	learn: 0.2795617	total: 121ms	remaining: 234ms
17:	learn: 0.2759657	total: 124ms	remaining: 221ms
18:	learn: 0.2724500	total: 128ms	remaining: 208ms
19:	learn: 0.2679366	total: 13

...

407:	learn: 0.0255466	total: 33.5s	remaining: 7.55s
408:	learn: 0.0254349	total: 33.6s	remaining: 7.47s
409:	learn: 0.0251968	total: 33.7s	remaining: 7.39s
410:	learn: 0.0250218	total: 33.7s	remaining: 7.3s
411:	learn: 0.0249109	total: 33.8s	remaining: 7.21s
412:	learn: 0.0247455	total: 33.9s	remaining: 7.13s
413:	learn: 0.0246445	total: 34s	remaining: 7.05s
414:	learn: 0.0244445	total: 34s	remaining: 6.96s
415:	learn: 0.0243248	total: 34.1s	remaining: 6.88s
416:	learn: 0.0241940	total: 34.2s	remaining: 6.8s
417:	learn: 0.0241893	total: 34.3s	remaining: 6.73s
418:	learn: 0.0239982	total: 34.4s	remaining: 6.65s
419:	learn: 0.0239746	total: 34.5s	remaining: 6.57s
420:	learn: 0.0238785	total: 34.5s	remaining: 6.47s
421:	learn: 0.0238670	total: 34.6s	remaining: 6.39s
422:	learn: 0.0237980	total: 34.7s	remaining: 6.31s
423:	learn: 0.0236520	total: 34.7s	remaining: 6.23s
424:	learn: 0.0235646	total: 34.8s	remaining: 6.13s
425:	learn: 0.0233903	total: 34.8s	remaining: 6.05s
426:	learn: 0.0233

67:	learn: 0.0374568	total: 5.75s	remaining: 36.6s
68:	learn: 0.0373255	total: 5.84s	remaining: 36.5s
69:	learn: 0.0360412	total: 5.94s	remaining: 36.5s
70:	learn: 0.0354376	total: 6.02s	remaining: 36.4s
71:	learn: 0.0345248	total: 6.09s	remaining: 36.2s
72:	learn: 0.0344783	total: 6.12s	remaining: 35.8s
73:	learn: 0.0341240	total: 6.19s	remaining: 35.6s
74:	learn: 0.0338807	total: 6.3s	remaining: 35.7s
75:	learn: 0.0330897	total: 6.39s	remaining: 35.6s
76:	learn: 0.0319082	total: 6.41s	remaining: 35.2s
77:	learn: 0.0314938	total: 6.49s	remaining: 35.1s
78:	learn: 0.0309659	total: 6.6s	remaining: 35.2s
79:	learn: 0.0301097	total: 6.68s	remaining: 35.1s
80:	learn: 0.0289944	total: 6.71s	remaining: 34.7s
81:	learn: 0.0284585	total: 6.8s	remaining: 34.7s
82:	learn: 0.0282694	total: 6.9s	remaining: 34.7s
83:	learn: 0.0277969	total: 6.99s	remaining: 34.6s
84:	learn: 0.0271072	total: 7.08s	remaining: 34.6s
85:	learn: 0.0263411	total: 7.1s	remaining: 34.2s
86:	learn: 0.0258381	total: 7.18s	re

227:	learn: 0.0010561	total: 18.7s	remaining: 22.3s
228:	learn: 0.0010211	total: 18.8s	remaining: 22.3s
229:	learn: 0.0010010	total: 18.9s	remaining: 22.2s
230:	learn: 0.0009872	total: 19s	remaining: 22.1s
231:	learn: 0.0009591	total: 19.1s	remaining: 22.1s
232:	learn: 0.0009495	total: 19.1s	remaining: 21.9s
233:	learn: 0.0009280	total: 19.2s	remaining: 21.8s
234:	learn: 0.0009006	total: 19.3s	remaining: 21.7s
235:	learn: 0.0008824	total: 19.3s	remaining: 21.6s
236:	learn: 0.0008559	total: 19.4s	remaining: 21.5s
237:	learn: 0.0008300	total: 19.5s	remaining: 21.4s
238:	learn: 0.0008174	total: 19.5s	remaining: 21.3s
239:	learn: 0.0008007	total: 19.6s	remaining: 21.2s
240:	learn: 0.0007858	total: 19.6s	remaining: 21.1s
241:	learn: 0.0007658	total: 19.7s	remaining: 21s
242:	learn: 0.0007579	total: 19.8s	remaining: 20.9s
243:	learn: 0.0007454	total: 19.9s	remaining: 20.8s
244:	learn: 0.0007339	total: 20s	remaining: 20.8s
245:	learn: 0.0007099	total: 20s	remaining: 20.6s
246:	learn: 0.000681

387:	learn: 0.0000295	total: 30.5s	remaining: 8.81s
388:	learn: 0.0000290	total: 30.6s	remaining: 8.74s
389:	learn: 0.0000283	total: 30.7s	remaining: 8.66s
390:	learn: 0.0000275	total: 30.8s	remaining: 8.57s
391:	learn: 0.0000268	total: 30.8s	remaining: 8.49s
392:	learn: 0.0000265	total: 30.9s	remaining: 8.42s
393:	learn: 0.0000261	total: 31s	remaining: 8.35s
394:	learn: 0.0000259	total: 31.2s	remaining: 8.28s
395:	learn: 0.0000253	total: 31.3s	remaining: 8.21s
396:	learn: 0.0000247	total: 31.3s	remaining: 8.12s
397:	learn: 0.0000240	total: 31.4s	remaining: 8.04s
398:	learn: 0.0000231	total: 31.5s	remaining: 7.96s
399:	learn: 0.0000227	total: 31.6s	remaining: 7.89s
400:	learn: 0.0000223	total: 31.8s	remaining: 7.86s
401:	learn: 0.0000223	total: 31.9s	remaining: 7.78s
402:	learn: 0.0000221	total: 32.1s	remaining: 7.72s
403:	learn: 0.0000218	total: 32.1s	remaining: 7.63s
404:	learn: 0.0000211	total: 32.2s	remaining: 7.55s
405:	learn: 0.0000210	total: 32.3s	remaining: 7.47s
406:	learn: 0.

KeyboardInterrupt: 

In [25]:
train_pool = Pool(X_train[selected_feat], y_train)
test_pool = Pool(X_test[selected_feat], y_test)

model = CatBoostRegressor(iterations=90, 
                          depth=2, 
                          learning_rate=0.1,
                          l2_leaf_reg=3,
                          loss_function='RMSE',
                          random_state= 1)

model.fit(train_pool)

pred_train = model.predict(train_pool)
pred_test = model.predict(test_pool)

print('train R2: {:.2f}'.format(r2_score(y_train, pred_train)))
print('test R2: {:.2f}'.format(r2_score(y_test, pred_test)))

0:	learn: 0.4412146	total: 1.6ms	remaining: 143ms
1:	learn: 0.4201270	total: 3.25ms	remaining: 143ms
2:	learn: 0.4012569	total: 4.81ms	remaining: 139ms
3:	learn: 0.3848804	total: 6.6ms	remaining: 142ms
4:	learn: 0.3713390	total: 9.23ms	remaining: 157ms
5:	learn: 0.3594281	total: 11.1ms	remaining: 155ms
6:	learn: 0.3482410	total: 12.7ms	remaining: 150ms
7:	learn: 0.3391776	total: 14.2ms	remaining: 145ms
8:	learn: 0.3311751	total: 15.6ms	remaining: 140ms
9:	learn: 0.3242939	total: 17.1ms	remaining: 137ms
10:	learn: 0.3177869	total: 18.7ms	remaining: 134ms
11:	learn: 0.3117262	total: 20.3ms	remaining: 132ms
12:	learn: 0.3063891	total: 22.3ms	remaining: 132ms
13:	learn: 0.3022172	total: 23.9ms	remaining: 130ms
14:	learn: 0.2984566	total: 25.8ms	remaining: 129ms
15:	learn: 0.2947368	total: 27.4ms	remaining: 127ms
16:	learn: 0.2909555	total: 29.2ms	remaining: 125ms
17:	learn: 0.2875154	total: 30.8ms	remaining: 123ms
18:	learn: 0.2846341	total: 32.4ms	remaining: 121ms
19:	learn: 0.2820412	tot

In [26]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train[selected_feat].columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

u10n_ivc_oiv_03_P95: 17.01283976283162
u10n_ivc_oiv_04_P95: 11.980120850326909
u10n_ivc_oiv_01_P95: 7.058903288738334
u10n_ivc_oiv_02_median: 5.746255394651023
u10n_ivc_oiv_07_median: 5.701298417361284
u10n_ivc_oiv_08_P95: 4.253679171730091
u100_res1od_median: 3.8280760151364395
u301_snr_id_median: 3.4763547084006388
u300_snr_id_median: 3.040896912349727
MCW_OD_median: 2.812684614420636
u100_res1md_median: 2.5009434256511844
u10n_ivc_oiv_04_median: 2.497267126570676
u10n_ivc_oiv_07_P95: 2.128411832873636
u101_mrwod_P95: 2.1129855967576634
u100_res1od_P5: 2.1034313696663323
u100_res1id_median: 2.002169645105333
u301_snr_id_std: 1.8648744163624011
u301_snr_id_P5: 1.692816608718993
OWC_OD_P95: 1.5513228512873292
MCW_OD_P95: 1.3363398899300871
u301_snr_md_P5: 1.3347987964629362
u304_ser_roller_ma_max_median: 1.1882364091598223
u307_rwser_rw_slp_nm_od_median: 1.1552587385781579
u101_mrwid_P5: 1.1549316055871162
u300_snr_id_std: 1.0297351596819415
u307_rwser_rw_slp_nm_od_P5: 0.91309664665404

In [52]:
pred_trains = pd.DataFrame({'Pred_pACC':pred_train}, index = X_train.index)
out_train = pd.concat([y_train, pred_trains, X_train], axis=1)

pred_tests = pd.DataFrame({'Pred_pACC':pred_test}, index = y_test.index)
out_test = pd.concat([y_test, pred_tests, X_test], axis=1)

out_train.to_csv('train_V7_G7.csv', index=True)
out_test.to_csv('test_V7_G7.csv', index=True)